In [1]:
import pandas as pd
from tqdm import tqdm
import polars as pl
from tqdm import tqdm
import joblib
import glob
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
def parsing_data(df: pd.DataFrame,target) -> pd.DataFrame:
    data = []
    for i in range(len(df)):
        for col in df.columns[1:]:
            data.append([df['Unnamed: 0'].iloc[i],float(col),df[col].iloc[i]])
    return pd.DataFrame(data,columns=['lat','long',f'{target}'])

In [3]:
def concat_data_read(path) -> pd.DataFrame:

    """
    We read all the files from the directory and concatenate them into one dataset

    """
    
    PATH = f'{path}/' #the path to the directory
    files = glob.glob(PATH + r'/**/*.csv', recursive=True)
    final_df = pd.DataFrame()
    for file in tqdm(files):
        if path != 'precipitation':
            date = file.split('.')[0].split('_')[3]
        else:
            date = file.split('.')[0].split('_')[1]
        df_file = pd.read_csv(file)
        data = parsing_data(df_file,path)
        data['date'] = date
        final_df = final_df._append(data)
    return final_df

In [4]:
#For each parameter values, specify the path to your directory and collect the datasets

In [5]:
vegetation = concat_data_read('vegetation_index')

100%|█████████████████████████████████████████| 257/257 [32:33<00:00,  7.60s/it]


In [6]:
#Let's save the dataset for training
vegetation.to_parquet("train_df.parquet", index=False, engine="pyarrow", compression="snappy")

In [7]:
#aero = concat_data_read('aerosol_depth')

In [8]:
#ground_moisture = concat_data_read('ground_moisture')

In [9]:
#land_temperature = concat_data_read('land_temperature')

In [10]:
#precipitation = concat_data_read('precipitation')

In [11]:
#shortwave_radiation = concat_data_read('shortwave_radiation')

In [12]:
#train = vegetation.merge(aero,how='left',on=['lat', 'long','date'])\
                 # .merge(ground_moisture,how='left',on=['lat', 'long','date'])\
                 # .merge(land_temperature,how='left',on=['lat', 'long','date'])\
                  #.merge(precipitation,how='left',on=['lat', 'long','date'])\
                 # .merge(shortwave_radiation,how='left',on=['lat', 'long','date'])

In [13]:
#train.to_parquet("train_df.parquet", index=False, engine="pyarrow", compression="snappy")